# Task 5: Quantum Graph Neural Network

We will be extending the idea of Task 2 to classify quark/gluon using GNN, but this time we do this using quantum circuits

The preprocessing part will be almost same.

Let us first import all the required libraries

In [1]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.nn import TransformerConv, global_mean_pool
from torch_cluster import knn_graph
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch_geometric.nn import TransformerConv
import pennylane as qml

## Load and Preprocess the dataset

As done in task 2, since the data set is very large so we will load the dataset in batches 

For preprocessing , this time I will be deleting all the particle data that has all 0 values and rest of the data is normalized

X : a 3D array of shape (num_jets, num_particles, num_features) where features are [pT, η, ϕ, PDG]. We keep only the first three columns.

y: a 1D array of jet-level labels

We will build a KNN graph 

Stores:<br>
    x: tensor of normalized continuous features (shape: [num_nodes, 3])<br>
    edge_index: graph connectivity<br>
    y: jet-level label

In [ ]:
class QGDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(QGDataset, self).__init__(root, transform, pre_transform)
        path = self.processed_paths[0]
        if os.path.exists(path):
            self.data, self.slices = torch.load(path)
        else:
            self.process()
            self.data, self.slices = torch.load(path)

    @property
    def raw_file_names(self):
        return glob.glob(os.path.join(self.raw_dir, '*.npz'))

    @property
    def processed_file_names(self):
        return ['data.pt']

    def process(self):
        data_list = []
        batch_size = 5000  # Process in chunks to avoid memory overload
        processed_path = self.processed_paths[0]

        for fpath in tqdm(self.raw_file_names, desc="Processing NPZ Files"):
            npz = np.load(fpath, mmap_mode='r')
            features = npz['X'].astype(np.float32)  
            labels = npz['y'].astype(np.int64)
            num_jets, num_particles, num_feats = features.shape

            for i in range(num_jets):
                x_np = features[i]
                mask = ~np.all(x_np == 0, axis=1)
                x_np = x_np[mask]
                if x_np.size == 0:
                    continue
                x_cont_np = x_np[:, :3]
                # Normalize features per jet
                mean = np.mean(x_cont_np, axis=0, keepdims=True)
                std = np.std(x_cont_np, axis=0, keepdims=True) + 1e-6
                x_cont_np = (x_cont_np - mean) / std

                x_cont = torch.tensor(x_cont_np, dtype=torch.float32)
                y = torch.tensor([labels[i]], dtype=torch.long)
                edge_index = knn_graph(x_cont, k=10, loop=False)

                data_obj = Data(x=x_cont, edge_index=edge_index, y=y)
                data_list.append(data_obj)

                if len(data_list) >= batch_size:
                    self._save_partial(data_list, processed_path)
                    data_list = []
        if data_list:
            self._save_partial(data_list, processed_path)

    def _save_partial(self, data_list, path):
        if os.path.exists(path):
            old_data, old_slices = torch.load(path)
            if isinstance(old_data, Data):
                old_data = [old_data]
            data_list = list(old_data) + data_list
        self.data, self.slices = self.collate(data_list)
        torch.save((self.data, self.slices), path)

## Quantum Circuit 

We will use 3 qubits to represent pT, η, ϕ 

`qml.AngleEmbedding`: Encodes input features as quantum angles.

`qml.BasicEntanglerLayers`: Applies trainable entangling layers.

`qml.expval(qml.PauliZ(i))`: Returns quantum measurement outputs.

Wrap the quantum circuit in a PyTorch Layer


In [3]:
n_qubits = 3  
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class QuantumLayer(nn.Module):
    def __init__(self, n_qubits, n_layers):
        super(QuantumLayer, self).__init__()
        weight_shapes = {"weights": (n_layers, n_qubits)}
        self.q_layer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)

    def forward(self, x):
        return self.q_layer(x)

## Hybrid Model

1. Graph Convolution (TransformerConv)
    Two layers:<br>
    First layer: TransformerConv with multi-head attention (4 heads).<br>
    Second layer: TransformerConv with single-head aggregation.<br>
    These layers extract graph-based features from the jet data.<br>

2. Global Pooling<br>
    Applies `global_mean_pool()` to aggregate node-level features into jet-level representations.

3. Quantum Feature Projection<br>
    Projects features to a 3-dimensional space (matching the quantum circuit).<br>
    Passes the features through the QuantumLayer.

4. Final Classification Layer<br>
    A linear layer maps quantum outputs to class logits.



In [6]:
class HybridTransformerQuantumModel(nn.Module):
    def __init__(self, in_channels=3, hidden_dim=64, out_dim=2, n_layers=3, n_qubits=3):
        super(HybridTransformerQuantumModel, self).__init__()
        # Layer 1
        self.conv1 = TransformerConv(in_channels, hidden_dim, heads=4, dropout=0.1)
        # Layer 2
        self.conv2 = TransformerConv(hidden_dim * 4, hidden_dim, heads=1, dropout=0.1)
        # Projection layer to make output equal to qubits
        self.feature_proj = nn.Linear(hidden_dim, n_qubits)
        self.q_layer = QuantumLayer(n_qubits, n_layers)
        self.lin = nn.Linear(n_qubits, out_dim)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index).relu()   # Output shape: (num_nodes, hidden_dim*4)
        x = self.conv2(x, edge_index).relu()   # Output shape: (num_nodes, hidden_dim)
        x = global_mean_pool(x, batch)         # Graph-level pooling -> shape: (num_graphs, hidden_dim)
        x = self.feature_proj(x)               # Project to (num_graphs, n_qubits=3)
        x = self.q_layer(x)                    # Quantum layer -> shape: (num_graphs, n_qubits)
        x = self.lin(x)                        # Final classification layer
        return x

## Train and Test Functions

Same as Task 2

In [7]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    for data in tqdm(loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(loader.dataset)

def test(model, loader, device):
    model.eval()
    y_true, y_pred = [], []
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            y_true.extend(data.y.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())
    return accuracy_score(y_true, y_pred)

## Main fxn

In [ ]:
def main():
    root_dir = 'qg_data'
    dataset = QGDataset(root=root_dir)
    print(f"Processed {len(dataset)} jets.")

    train_size = int(0.8 * len(dataset))
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = QGNN(in_channels=3, hidden_dim=128, out_dim=2, n_layers=3, n_qubits=3).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

    best_acc = 0
    for epoch in range(1, 51):
        loss = train(model, train_loader, optimizer, device)
        acc = test(model, test_loader, device)
        print(f"Epoch {epoch:02d}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}")
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), "qgnn_improved_best.pth")

    print("Best Test Accuracy:", best_acc)

if __name__ == "__main__":
    main()


C:\Users\arnav\AppData\Local\Temp\ipykernel_22224\1247508449.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(path)
C:\Users\arnav\an

Processed 5000 jets.


Training: 100%|██████████| 125/125 [00:10<00:00, 11.94it/s]


Epoch 01, Loss: 0.6887, Test Accuracy: 0.6750


Training: 100%|██████████| 125/125 [00:09<00:00, 12.83it/s]


...

Epoch 48, Loss: 0.5494, Test Accuracy: 0.7300


Training: 100%|██████████| 125/125 [00:09<00:00, 12.55it/s]


Epoch 49, Loss: 0.5497, Test Accuracy: 0.7250


Training: 100%|██████████| 125/125 [00:09<00:00, 13.02it/s]


Epoch 50, Loss: 0.5443, Test Accuracy: 0.7310
Best Test Accuracy: 0.734


Due to memory issue I cannot increase its accuracy as by increasing any parameter or fine tuning leads to Out of Memory Error

This model gives accuracy of 73.4% which is less than the models we have done in task 2 because of the issues in Computational Power, maybe if I had more computational power, the quantum model would have performed better.